In [59]:
import torch
from torch import nn

### Basic Operation (channels = 1, stride = 1, padding = 0)

In [16]:
def transposed_conv(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            for ki in range(h):
                for kj in range(w):
                    Y[i + ki, j + kj] += X[i, j]*K[ki, kj]
    return Y


In [17]:
X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
Y = transposed_conv(X, K)

X, K = X.reshape(1, 1, 2, 2,), K.reshape(1, 1, 2, 2)
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)
tconv.weight.data = K
Y_gold = tconv(X)

print(Y)
print(Y_gold)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])
tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<SlowConvTranspose2DBackward>)


### Convolution Matrix for Transposed Convolution           

In [2]:
import numpy as np 

C:\Users\dstum\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\dstum\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\dstum\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


output size = s(n - 1) + k -2p<br/>

s: stride<br/>
n: input height/width<br/>
k: kernel size<br/>
p: padding<br/>

In [34]:
def conv_matrix(k,stride=1,input_size=128, padding=0):
    op_size = stride*(input_size-1) + k.shape[1] - 2*padding
    m_rows, m_cols = op_size, op_size
    k_rows, k_cols = k.shape[1], k.shape[1]
    
    rows_out = m_rows - k_rows + 1
    cols_out = m_cols - k_cols + 1
    
    v = np.zeros((rows_out*cols_out, m_rows, m_cols))
    
    for r in range(rows_out):
        for c in range(cols_out):
            for kr in range(k_rows):
                for kc in range(k_cols):
                    v[r * cols_out + c][r + kr, c + kc] = k[kr, kc]
    
    v = v.reshape((rows_out*cols_out), -1)
    return v, op_size

def column_vector(x):
    return x.flatten().reshape(-1, 1)
    

In [48]:
np.random.seed(0)
K = np.random.randint(1, 10, size=(3,3))
X = np.random.randint(1,5, size = (6, 6))

W, ops = conv_matrix(K,input_size=X.shape[0],padding=0, stride=1)
X = column_vector(X)
print(W.T.shape)
print(X.shape)
#Y = W.T @ X
#Y = Y.reshape((ops, ops))

#print(ops)
#print(Y)


(64, 36)
(36, 1)


[[14. 31. 25. 15.]
 [22. 37. 43. 29.]
 [12. 24. 43. 29.]
 [ 4.  8. 17.  7.]]


## Transposed Conv.

output size = s(n - 1) + k -2p<br/>

s: stride<br/>
n: input height/width<br/>
k: kernel size<br/>
p: padding<br/>

Increase p -> Decrease output size

In [74]:
X = torch.rand(64).view(1,1,8,8)

conv_t = nn.ConvTranspose2d(in_channels=1,
                            out_channels=1,
                            stride=5,
                            kernel_size=(3,3),
                            padding=1
                            )
op_padding = conv_t._output_padding(X,
                       stride=conv_t.stride,
                       padding=conv_t.padding,
                       kernel_size=conv_t.kernel_size,
                       output_size=None
                       )
print(op_padding)
Y = nn.functional.conv_transpose2d(X,conv_t.weight, conv_t.bias, conv_t.stride, conv_t.padding, op_padding, conv_t.groups, conv_t.dilation)
print(Y.size())
print(conv_t(X).size())

(0, 0)
torch.Size([1, 1, 36, 36])
torch.Size([1, 1, 36, 36])


In [56]:

stride = 4
ip_padding = 0
op_padding = 1
filter_size = 3
input_size = 4

# create input
X = np.linspace(1, 16, 16).reshape((input_size, input_size))

# create kernel
K = np.ones((filter_size,filter_size))

# get output size
op_size = stride*(input_size-1) + filter_size - 2*ip_padding + op_padding

# create output image init to zeros
Y = np.zeros((op_size,op_size))

for i in range(input_size):
    for j in range(input_size):
        for ki in range(filter_size):
            for kj in range(filter_size):
                
        


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [69]:
def _filter_to_mat(f):
    """
    Converts a filter to matrix form
    :param f: The filter (num_filters, depth, height, width)
    :return: The matrix form of the filter which can be multiplied
    """
    return f.reshape(f.shape[0], -1).T

def _backward_im_to_rows(top_grad, inp_shape, filter_shape, dilation, stride, dilated_shape, res_shape):
    """
    Gradient transformation for the im2rows operation
    :param top_grad: The grad from the next layer
    :param inp_shape: The shape of the input image
    :param filter_shape: The shape of the filter (num_filters, depth, height, width)
    :param dilation: The dilation for the filter
    :param stride: The stride for the filter
    :param dilated_shape: The dilated shape of the filter
    :param res_shape: The shape of the expected result
    :return: The reformed gradient of the shape of the image
    """
    dilated_rows, dilated_cols = dilated_shape
    num_rows, num_cols = res_shape
    res = np.zeros(inp_shape, dtype=top_grad.dtype)
    top_grad = top_grad.reshape(
        (top_grad.shape[0], top_grad.shape[1], filter_shape[1], filter_shape[2], filter_shape[3]))
    for it in range(num_rows * num_cols):
        i = it // num_rows
        j = it % num_rows
        res[:, :, i * stride[0]:i * stride[0] + dilated_rows:dilation,
            j * stride[1]:j * stride[1] + dilated_cols:dilation] += top_grad[:, it, :, :, :]
    return res

def transposed_conv2d(image, filters, dilation, stride):
    """
    Perform a transposed convolution, which can upscale the image.
    :param image: The input image to upscale
    :param filters: The filters for this operation
    :param dilation: The dilation factor for the filters
    :param stride: The stride for the *forward* convolution
    :return: The return upscaled image
    """
    filter_shape = filters.shape
    im_shape = image.shape
    dilated_shape = ((filter_shape[2] - 1) * dilation + 1,
                     (filter_shape[3] - 1) * dilation + 1)
    res_shape = (im_shape[2] - 1) * stride[0] + \
        dilated_shape[0], (im_shape[3] - 1) * stride[1] + dilated_shape[1]
    image_mat = image.reshape(
        (image.shape[0], image.shape[1], -1)).transpose((0, 2, 1))
    filtmat = _filter_to_mat(filters)
    res_mat = image_mat.dot(filtmat.T)
    return _backward_im_to_rows(res_mat, (image.shape[0], filters.shape[1], *res_shape), filters.shape, dilation,
                                stride, dilated_shape, im_shape[2:])


In [70]:
X = torch.rand(64).view(1,1,8,8)

conv_t = nn.ConvTranspose2d(in_channels=1,
                            out_channels=1,
                            stride=5,
                            kernel_size=(3,3),
                            padding=1
                            )
conv_t.weight = torch.nn.Parameter(torch.ones_like(conv_t.weight))
conv_t(X)

K = torch.ones((1,1,3,3))
Y = transposed_conv2d(image=X, filters=K, dilation=0, stride=1)

TypeError: 'int' object is not subscriptable